**Juan Pablo Ocampo Santana**

In [2]:
# Uninstall and reinstall packages to fix compatibility issues
!pip uninstall numpy pandas -y
!pip install numpy pandas facenet-pytorch torch torchvision pillow

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.3.2
Uninstalling pandas-2.3.2:
  Successfully uninstalled pandas-2.3.2
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu1

In [2]:
# 1) Instalación de dependencias
# Asegúrate de tener estas librerías instaladas
# pip install facenet-pytorch torch torchvision numpy pandas pillow

# 2) Importaciones
import torch
import numpy as np
import pandas as pd
import itertools
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import os

# 3) Obtener la lista de imágenes
# Reemplaza 'path/a/tus/imagenes' con la ruta a la carpeta donde tienes las 3 fotos.
# Asegúrate de que solo haya 3 imágenes en la carpeta para evitar errores.
images_dir = '/content/Fotos'
filenames = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if len(filenames) != 3:
    raise ValueError(f"Debes tener exactamente 3 imágenes en la carpeta '{images_dir}'. Se encontraron: {len(filenames)}")

# 4) Preparar detector y red de embeddings
# Elige el dispositivo de procesamiento, GPU si está disponible, sino CPU.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# MTCNN: Detector de rostros y alineador.
# Ajusta el 'margin' si los rostros están muy cerca del borde de la foto.
mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=device)

# InceptionResnetV1: Modelo que genera los 'embeddings' o vectores de características del rostro.
# 'vggface2' es un modelo pre-entrenado que funciona muy bien para esta tarea.
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def load_and_embed(path):
    """
    - Abre la imagen desde una ruta.
    - Detecta y alinea el rostro.
    - Obtiene un embedding (vector de 512 dimensiones) que representa el rostro.
    """
    img = Image.open(path).convert('RGB')
    face_tensor = mtcnn(img)
    if face_tensor is None:
        raise ValueError(
            f"No se detectó rostro en: {path}. "
            "Asegúrate de que la cara esté visible y bien iluminada."
        )
    face_tensor = face_tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        emb = resnet(face_tensor).cpu().numpy().flatten()
    return emb

# 5) Obtener embeddings de las 3 imágenes
print("Procesando imágenes y generando embeddings...")
embeddings = {}
for name in filenames:
    embeddings[os.path.basename(name)] = load_and_embed(name)

# 6) Funciones de distancia
# La distancia nos ayuda a cuantificar qué tan "parecidos" son dos rostros.
def euclidean(a, b):
    return float(np.linalg.norm(a - b))

def cosine_distance(a, b):
    # La distancia de coseno mide el ángulo entre los vectores.
    # Un valor bajo (cercano a 0) indica alta similitud.
    num = float(np.dot(a, b))
    den = float(np.linalg.norm(a) * np.linalg.norm(b))
    return 1.0 - (num / den)

# 7) Comparar las 3 parejas posibles
pairs = list(itertools.combinations(embeddings.keys(), 2))
rows = []
for (fa, fb) in pairs:
    ea, eb = embeddings[fa], embeddings[fb]
    rows.append({
        "Par de imágenes": f"{fa} ↔ {fb}",
        "Distancia Euclídea (↓ = más parecidas)": euclidean(ea, eb),
        "Distancia de Coseno (↓ = más parecidas)": cosine_distance(ea, eb),
    })

df = pd.DataFrame(rows).sort_values(by="Distancia Euclídea (↓ = más parecidas)")
print("\n=== Resultados de la comparación ===")
print(df)

# 8) Conclusión: par más probable misma persona
# El par con la menor distancia euclídea es el más probable de ser la misma persona.
best_pair = df.iloc[0]["Par de imágenes"]
print(f"\n✅ PAREJA MÁS PROBABLE (misma persona): {best_pair}")

Procesando imágenes y generando embeddings...

=== Resultados de la comparación ===
                        Par de imágenes  \
0    Johnny_Depp.jpg ↔ Johnny_Depp2.jpg   
1   Johnny_Depp.jpg ↔ Robert_Downey.jpg   
2  Johnny_Depp2.jpg ↔ Robert_Downey.jpg   

   Distancia Euclídea (↓ = más parecidas)  \
0                                0.640329   
1                                1.256776   
2                                1.289975   

   Distancia de Coseno (↓ = más parecidas)  
0                                 0.205010  
1                                 0.789743  
2                                 0.832018  

✅ PAREJA MÁS PROBABLE (misma persona): Johnny_Depp.jpg ↔ Johnny_Depp2.jpg
